# Notebook to download data

This notebook imports data for COVID related analysis from a number of online sources.

First of all, we need to import a few useful libraries.

In [10]:
import pandas as pd
import numpy as np
import os
from pandasdmx import Request 
import eurostat
from concurrent.futures import ThreadPoolExecutor
import time
from datetime import datetime

Make sure to have all packages. If not, type "pip install --namepackage--" in the console or in the terminal. We can now start to get the data. 

Define function to get the downloaded data in empty lists.

## Data From [World Population Prospects](https://population.un.org/wpp/Download/Standard/CSV/) and [Johns Hopkins](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series)

John Hopkins provides daily data on **confirmed cases**, **deaths** and **recovered** for many countries in the world. Data can be freely accessed through GitHub. We can also get data from the UN.

In [11]:
start_time = time.time()

def data_JH(url, container):
    ind = 0
    for i in url:
        if ind <= len(url):
            container.append([url[ind][0],pd.read_csv(url[ind][1], index_col = 0, parse_dates = [0])])
        ind += 1

Declare data to download.

In [12]:
url_WPP = [['WPP_tot_pop', 'https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_TotalPopulationBySex.csv'],
          ['WPP_pop_age_sec', 'https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_PopulationByAgeSex_Medium.csv'],
          ['WPP_fertility', 'https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/CSV_FILES/WPP2019_Fertility_by_Age.csv']]

url_JH = [['JH_confirmed', 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'],
            ['JH_death', 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'],
            ['JH_recovered', 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv']]


## Data From [Eurostat](https://ec.europa.eu/eurostat/web/population-demography-migration-projections/data/database)

The library Eurostat allows to get demographics data in one line of code. We download data about population as well as national accounts. Data need to be cleaned for later usage.

In [13]:
def data_Eurostat(url, container):
    ind = 0
    for i in url:
        if ind <= len(url):
            container.append([url[ind][0], eurostat.get_data_df(url[ind][1])])
        ind += 1

Define the variables according to their codes on eurostat.

In [14]:
Eurostat_code = [['EU_pop', 'demo_r_gind3'], ['EU_gdp', 'nama_10_gdp'], ['EU_cons', 'nama_10_fcs'], ['EU_trade', 'nama_10_exi'], ['EU_short_rate', 'irt_st_a'],
                ['EU_long_rate', 'irt_lt_gby10_a'], ['EU_unemp', 'une_rt_a'], ['EU_inv', 'nama_10_an6']]

## Data From [Oxford COVID19 Government Response Tracker](https://www.bsg.ox.ac.uk/research/research-projects/oxford-covid-19-government-response-tracker)

This dataset records a number of different measures that governments are taking to face the COVID19 outbreak around the world.

In [15]:
def data_Ox(url, container):
    ind = 0
    for i in url:
        if ind <= len(url):
            container.append([url[ind][0],pd.read_excel(url[ind][1], index_col = 0, parse_dates = [0])])
        ind += 1

In [16]:
url_Ox = [['OX_govt_responses', 'https://www.bsg.ox.ac.uk/sites/default/files/OxCGRT_Download_latest_data.xlsx']]

## Data From [Sciensano - Belgium](https://epistat.wiv-isp.be/covid/)

This dataset contains detailed information for Belgium.

In [17]:
def data_Belgium(url, container):
    ind = 0
    for i in url:
        if ind <= len(url):
            container.append([url[ind][0],pd.read_csv(url[ind][1], index_col = 0, parse_dates = [0], encoding = "ISO-8859-1")])
        ind += 1

In [18]:
url_Belgium = [['Confirmed_cases_by_date_age_sex_and_province', 'https://epistat.sciensano.be/Data/COVID19BE_CASES_AGESEX.csv'],
               ['Confirmed_cases_by_date_and_municipality', 'https://epistat.sciensano.be/Data/COVID19BE_CASES_MUNI.csv'],
               ['Cumulative_number_of_confirmed_cases_by_municipality', 'https://epistat.sciensano.be/Data/COVID19BE_CASES_MUNI_CUM.csv'],
               ['Hospitalisations_by_date_and_provinces', 'https://epistat.sciensano.be/Data/COVID19BE_HOSP.csv'],
               ['Mortality_by_date_age_sex_and_province', 'https://epistat.sciensano.be/Data/COVID19BE_MORT.csv'],
               ['Total_number_of_tests_performed_by_date', 'https://epistat.sciensano.be/Data/COVID19BE_tests.csv']]

## Download Data

We can now download all the data. To this end, we create empty containers and run the functions.

In [109]:
JH  = []
WPP = []
EU  = []
OX  = []
BE  = []

with ThreadPoolExecutor(max_workers = 5) as e:
    e.submit(data_JH(url_JH, JH))
    e.submit(data_JH(url_WPP, WPP))
    e.submit(data_Eurostat(Eurostat_code, EU))
    e.submit(data_Ox(url_Ox, OX))
    e.submit(data_Belgium(url_Belgium, BE))

## Clean Data

This section is dedicated to do some preliminary data cleaning. First, we reshape JH data from wide to long.

In [112]:
def reshape_JH(JH,JH_reshaped):
    
    for j in range(len(JH)):
    
        # Get and clean an entry of the JH list
        tmp = pd.DataFrame(JH[j][1])
        tmp = tmp.drop(['Lat', 'Long'], axis=1)

        # Rename columns
        tmp.columns = tmp.columns.str.replace('/','')
        new_names = [(i,'Date' + i) for i in tmp.iloc[:, 1:].columns.values]
        tmp.rename(columns = dict(new_names), inplace=True)

        # Rename rows
        tmp.index = pd.Series(tmp.index).replace(np.nan, 'Unique')

        # Create a new ID = row name + first column
        tmp['idx'] = tmp.index
        tmp = tmp.set_index(['CountryRegion', 'idx'])
        tmp['idx'] = tmp.index
        tmp = tmp.reset_index(drop=True)

        # Reshape wide to long
        tmp = pd.wide_to_long(tmp, ["Date"], i="idx", j="date")
        tmp = tmp.rename(columns = {'Date':'occurrences'})

        # Clean index
        idx = []
        cou = []
        reg = []
        date = []
        for k in range(len(tmp.index)):
            idx.append(list(tmp.index[k])[0])
            date.append(list(tmp.index[k])[1])
        
        # Get country and region identifier
        for i in range(len(idx)):
            cou.append(idx[i][0])
            reg.append(idx[i][1])
            
        # Finalize dataset
        tmp['country'] = cou  
        tmp['region'] = reg  
        tmp['date'] = [str(i)+'20' for i in date]
        tmp['date'] = pd.to_datetime(tmp['date'], format='%m%d%Y', errors='coerce')
        tmp = tmp.reset_index(drop=True)
        tmp = tmp[["country", "region", "date", "occurrences"]]

        # Store output in a new list
        JH_reshaped.append([JH[j][0], tmp])
        
    #Return
    return(JH_reshaped)

In [116]:
JH_reshaped = []
JH_reshaped = reshape_JH(JH,JH_reshaped)

## Save all data

Run the lines below **only** if you need to save data locally. The default path of the function is your current working directory. If another path is specified, the function will create the folders in that path if they do not already exist.

In [117]:
def store_data(path = os.getcwd(), JH_data = True, WPP_data = True, EU_data = True, Ox_data = True, BE_data = True):
    os.makedirs(path, exist_ok = True)
    if JH_data:
        ind = 0
        for i in JH_reshaped:
            JH_reshaped[ind][1].to_csv(path + JH_reshaped[ind][0] + '.csv', index = False)
            ind += 1
    if WPP_data:
        ind = 0
        for i in WPP:
            WPP[ind][1].to_csv(path + WPP[ind][0] + '.csv', index = False)
            ind += 1
    if EU_data:
        ind = 0
        for i in EU:
            EU[ind][1].to_csv(path + EU[ind][0] + '.csv', index = False)
            ind += 1
    if Ox_data:
        ind = 0
        for i in OX:
            OX[ind][1].to_csv(path + OX[ind][0] + '.csv', index = False)
            ind += 1
    if BE_data:
        ind = 0
        for i in BE:
            BE[ind][1].to_csv(path + BE[ind][0] + '.csv', index = False)
            ind += 1

Save the data in a folder called *files* inside your current working directory.

In [118]:
store_data(os.getcwd() + "/files/")
end_time = time.time()
print(f"Total time: {end_time - start_time}")

Total time: 2354.8951001167297
